# \[FRAUD\] 데이터 (9.13_df50 edge다르게)

김보람  
2023-09-13

# imports

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import networkx as nx
import sklearn
import xgboost as xgb

# sklearn
from sklearn import model_selection # split함수이용
from sklearn import ensemble # RF,GBM
from sklearn import metrics
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB

# embedding 
from node2vec import Node2Vec
from node2vec.edges import HadamardEmbedder, AverageEmbedder, WeightedL1Embedder, WeightedL2Embedder

# gnn
import torch
import torch_geometric
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

In [6]:
def build_graph_bipartite(df_input, graph_type=nx.Graph()):
    df=df_input.copy()
    mapping={x:node_id for node_id, x in enumerate(set(df["cc_num"].values.tolist()+\
                                                      df["merchant"].values.tolist()))}
    
    df["from"]=df["cc_num"].apply(lambda x:mapping[x])  #엣지의 출발점
    df["to"]=df["merchant"].apply(lambda x:mapping[x])  #엣지의 도착점
    
    df = df[['from', 'to', "amt", "is_fraud"]].groupby(['from','to']).agg({"is_fraud":"sum","amt":"sum"}).reset_index()
    df["is_fraud"]=df["is_fraud"].apply(lambda x:1 if x>0 else 0)
    
    G=nx.from_edgelist(df[["from","to"]].values, create_using=graph_type)
    
    nx.set_edge_attributes(G,{(int(x["from"]),int(x["to"])):x["is_fraud"] for idx, x in df[["from","to","is_fraud"]].iterrows()}, "label")  #엣지 속성 설정,각 속성의 사기 여부부     
    nx.set_edge_attributes(G,{(int(x["from"]),int(x["to"])):x["amt"] for idx,x in df[["from","to","amt"]].iterrows()}, "weight") # 엣지 속성 설정, 각 엣지의 거래 금액

    return G


def build_graph_tripartite(df_input, graph_type=nx.Graph()):
    df=df_input.copy()
    mapping={x:node_id for node_id, x in enumerate(set(df.index.values.tolist() + 
                                                       df["cc_num"].values.tolist() +
                                                       df["merchant"].values.tolist()))}
    df["in_node"]= df["cc_num"].apply(lambda x: mapping[x])
    df["out_node"]=df["merchant"].apply(lambda x:mapping[x])
    
        
    G=nx.from_edgelist([(x["in_node"], mapping[idx]) for idx, x in df.iterrows()] +\
                        [(x["out_node"], mapping[idx]) for idx, x in df.iterrows()], create_using=graph_type)
    
    nx.set_edge_attributes(G,{(x["in_node"], mapping[idx]):x["is_fraud"] for idx, x in df.iterrows()}, "label")     
    nx.set_edge_attributes(G,{(x["out_node"], mapping[idx]):x["is_fraud"] for idx, x in df.iterrows()}, "label")   
    nx.set_edge_attributes(G,{(x["in_node"], mapping[idx]):x["amt"] for idx, x in df.iterrows()}, "weight")  
    nx.set_edge_attributes(G,{(x["out_node"], mapping[idx]):x["amt"] for idx, x in df.iterrows()}, "weight")

    return G
    
    
def down_sample_textbook(df):
    df_majority = df[df.is_fraud==0].copy()
    df_minority = df[df.is_fraud==1].copy()
    df_maj_dowsampled = sklearn.utils.resample(df_majority, n_samples=len(df_minority), replace=False, random_state=42)
    df_downsampled = pd.concat([df_minority, df_maj_dowsampled])
    return df_downsampled

def embedding(Graph):
    # Graph -> X (feature)
    _edgs = list(Graph.edges)
    subGraph = Graph.edge_subgraph([_edgs[x] for x in range(len(Graph.edges))]).copy()
    subGraph.add_nodes_from(list(set(Graph.nodes) - set(subGraph.nodes)))    
    embedded = AverageEmbedder(Node2Vec(subGraph, weight_key='weight').fit(window=10).wv)
    X = [embedded[str(_edgs[x][0]), str(_edgs[x][1])] for x in range(len(Graph.edges))]
    # Graph -> y (label)
    y = np.array(list(nx.get_edge_attributes(Graph, "label").values()))
    return X,y 

def anal(df):
    Graph = build_graph_bipartite(df)
    X,XX,y,yy = embedding(Graph)
    lrnr = RandomForestClassifier(n_estimators=100, random_state=42) 
    lrnr.fit(X,y)
    yyhat = lrnr.predict(XX)
    df = pd.DataFrame({
        'acc':[sklearn.metrics.accuracy_score(yy,yyhat)], 
        'pre':[sklearn.metrics.precision_score(yy,yyhat)], 
        'rec':[sklearn.metrics.recall_score(yy,yyhat)],
        'f1':[sklearn.metrics.f1_score(yy,yyhat)]}
    )    
    return df

def our_sampling1(df):
    cus_list = set(df.query('is_fraud==1').cc_num.tolist())
    return df.query("cc_num in @ cus_list")

In [7]:
fraudTrain = pd.read_csv("~/Desktop/fraudTrain.csv").iloc[:,1:]

In [8]:
fraudTrain = fraudTrain.assign(trans_date_trans_time= list(map(lambda x: pd.to_datetime(x), fraudTrain.trans_date_trans_time)))
fraudTrain

## GNN시도

In [9]:
_df1 = fraudTrain[fraudTrain["is_fraud"] == 0].sample(frac=0.20, random_state=42)
_df2 = fraudTrain[fraudTrain["is_fraud"] == 1]
df02 = pd.concat([_df1,_df2])
df02.shape

In [10]:
df50 = down_sample_textbook(df02)
df50.shape

In [11]:
df50 = df50.reset_index()

In [12]:
N = len(df50)

------------------------------------------------------------------------

### tr/test

In [13]:
df50_tr,df50_test = sklearn.model_selection.train_test_split(df50, random_state=42)

In [14]:
df50_tr.shape, df50_test.shape

In [15]:
train_mask = [i in df50_tr.index for i in range(N)]
test_mask = [i in df50_test.index for i in range(N)]

In [16]:
train_mask = np.array(train_mask)
test_mask = np.array(test_mask)

In [17]:
train_mask.sum(), test_mask.sum()

In [18]:
train_mask.shape, test_mask.shape

------------------------------------------------------------------------

### edge_index 설정

In [20]:
def compute_time_difference(group):
    n = len(group)
    result = []
    for i in range(n):
        for j in range(n):
            time_difference = abs(group.iloc[i].trans_date_trans_time.value - group.iloc[j].trans_date_trans_time.value)
            result.append([group.iloc[i].name, group.iloc[j].name, time_difference])
    return result

In [21]:
groups = df50.groupby('cc_num')
edge_index_list_plus = [compute_time_difference(group) for _, group in groups]
edge_index_list_plus_flat = [item for sublist in edge_index_list_plus for item in sublist]
edge_index_list_plus_nparr = np.array(edge_index_list_plus_flat)
np.save('edge_index_list_plus50.npy', edge_index_list_plus_nparr)

In [22]:
# edge_index_list_plus = []
# for i in range(N):
#     for j in range(N):
#         if df50['cc_num'][i] != df50['cc_num'][j]:  # cc_num 값이 다르다면
#             time_difference = 0
#         else:
#             time_difference = (df50['trans_date_trans_time'][i] - df50['trans_date_trans_time'][j]).total_seconds()
#         edge_index_list_plus.append([i, j, time_difference])
#         np.save('edge_index_list_plus50.npy', edge_index_list_plus)

# # edge_index_list_plus = np.load('edge_index_list_plus.npy')

In [38]:
edge_index = np.array(edge_index_list_plus_nparr)

In [39]:
edge_index.shape

In [40]:
weight = (np.exp(-edge_index[:,2]/theta) != 1)*(np.exp(-edge_index[:,2]/theta))
weight

In [41]:
edge_index = np.column_stack((edge_index, weight))
edge_index = np.delete(edge_index, 2, axis=1)

In [42]:
edge_index

In [43]:
edge_index.shape

In [44]:
edge_index = edge_index.tolist()

In [48]:
mean_ = np.array(edge_index)[:,2].mean()

`-` median

In [49]:
medi_ = np.median(np.array(edge_index)[:,2])

In [50]:
selected_edges = [(int(row[0]), int(row[1])) for row in edge_index if row[2] > medi_]
edge_index_selected = torch.tensor(selected_edges, dtype=torch.long).t()
edge_index_selected.shape

------------------------------------------------------------------------

### data설정(x, edge_index, y)

In [52]:
x = torch.tensor(df50['amt'], dtype=torch.float).reshape(-1,1)

In [53]:
y = torch.tensor(df50['is_fraud'],dtype=torch.int64)

In [54]:
data = torch_geometric.data.Data(x=x, edge_index = edge_index_selected, y=y, train_mask = train_mask, test_mask = test_mask)

In [55]:
data

------------------------------------------------------------------------

### gnn

In [56]:

class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(1, 16)
        self.conv2 = GCNConv(16,2)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)

In [57]:
model = GCN()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
model.train()
for epoch in range(400):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

In [58]:
model.eval()
pred = model(data).argmax(dim=1)
correct = (pred[data.test_mask] == data.y[data.test_mask]).sum()
acc = int(correct) / int(np.array(data.test_mask).sum())
print(f'Accuracy: {acc:.4f}')

Accuracy: 0.8768

In [59]:
predicted_labels = pred[data.test_mask]
true_labels = data.y[data.test_mask]


In [60]:
precision = precision_score(true_labels, predicted_labels, average='macro')
recall = recall_score(true_labels, predicted_labels, average='macro')
f1 = f1_score(true_labels, predicted_labels, average='macro')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')

Precision: 0.8799
Recall: 0.8763
F1 Score: 0.8764

------------------------------------------------------------------------

# 정리

| 구분  | Train   | Test      | 모형          | 설명변수 | 비고 |
|-------|---------|-----------|---------------|----------|------|
| 분석1 | df50_tr | df50_test | GNN           | amt      |      |
| 분석2 | df50_tr | df50_test | 로지스틱 회귀 | amt      |      |
| 분석3 | df50_tr | df50_test | SVM           | amt      |      |
| 분석4 | df50_tr | df50_test | 랜덤포레스트  | amt      |      |

## 분석2(로지스틱 회귀)

In [31]:
X = np.array(df50_tr.loc[:,['amt']])
XX = np.array(df50_test.loc[:,['amt']])
y = np.array(df50_tr.is_fraud)
yy = np.array(df50_test.is_fraud)

In [34]:
lrnr = sklearn.linear_model.LogisticRegression()

In [35]:
lrnr.fit(X,y)

In [36]:
#thresh = y.mean()
#yyhat = (lrnr.predict_proba(XX)> thresh)[:,-1]
yyhat = lrnr.predict(XX) 

In [37]:
metrics = [sklearn.metrics.accuracy_score,
           sklearn.metrics.precision_score,
           sklearn.metrics.recall_score,
           sklearn.metrics.f1_score]

In [38]:
_results2= pd.DataFrame({m.__name__:[m(yy,yyhat).round(6)] for m in metrics},index=['분석2'])
_results2

## 분석3(서포트 벡터 머신)

In [40]:
X = np.array(df50_tr.loc[:, ['amt']])
XX = np.array(df50_test.loc[:, ['amt']])
y = np.array(df50_tr.is_fraud)
yy = np.array(df50_test.is_fraud)

In [41]:
lrnr = SVC(kernel='linear')  
lrnr.fit(X,y)
yyhat = lrnr.predict(XX)


In [42]:
metrics = [sklearn.metrics.accuracy_score,
           sklearn.metrics.precision_score,
           sklearn.metrics.recall_score,
           sklearn.metrics.f1_score]

In [43]:
_results3= pd.DataFrame({m.__name__:[m(yy,yyhat).round(6)] for m in metrics},index=['분석3'])
_results3

## 분석4(랜덤 포레스트)

In [46]:
X = np.array(df50_tr.loc[:, ['amt']])
XX = np.array(df50_test.loc[:, ['amt']])
y = np.array(df50_tr.is_fraud)
yy = np.array(df50_test.is_fraud)

In [47]:
lrnr = RandomForestClassifier()  
lrnr.fit(X, y)
yyhat = lrnr.predict(XX)

In [49]:
metrics = [sklearn.metrics.accuracy_score,
           sklearn.metrics.precision_score,
           sklearn.metrics.recall_score,
           sklearn.metrics.f1_score]

In [50]:
_results4= pd.DataFrame({m.__name__:[m(yy,yyhat).round(6)] for m in metrics},index=['분석4'])
_results4

## 분석5(부스팅)

In [54]:
X = np.array(df50_tr.loc[:, ['amt']])
XX = np.array(df50_test.loc[:, ['amt']])
y = np.array(df50_tr.is_fraud)
yy = np.array(df50_test.is_fraud)

In [55]:
lrnr = xgb.XGBClassifier()  
lrnr.fit(X, y)
yyhat = lrnr.predict(XX)

In [56]:
metrics = [sklearn.metrics.accuracy_score,
           sklearn.metrics.precision_score,
           sklearn.metrics.recall_score,
           sklearn.metrics.f1_score]

In [57]:
_results5= pd.DataFrame({m.__name__:[m(yy,yyhat).round(6)] for m in metrics},index=['분석5'])
_results5

## 분석6(Naive Bayes)

In [59]:
X = np.array(df50_tr.loc[:, ['amt']])
XX = np.array(df50_test.loc[:, ['amt']])
y = np.array(df50_tr.is_fraud)
yy = np.array(df50_test.is_fraud)

In [60]:
lrnr = GaussianNB() 
lrnr.fit(X, y)
yyhat = lrnr.predict(XX)

In [61]:
metrics = [sklearn.metrics.accuracy_score,
           sklearn.metrics.precision_score,
           sklearn.metrics.recall_score,
           sklearn.metrics.f1_score]

In [62]:
_results6= pd.DataFrame({m.__name__:[m(yy,yyhat).round(6)] for m in metrics},index=['분석6'])
_results6

In [1]:
!git add .

In [2]:
!git commit -m .

[main f9769619] .
 8 files changed, 8800 insertions(+), 2514 deletions(-)
 create mode 100644 "posts/GNN/FRAUD/.ipynb_checkpoints/230822 \353\215\260\354\235\264\355\204\260(6, df02)-Copy1-checkpoint.ipynb"
 create mode 100644 "posts/GNN/FRAUD/.ipynb_checkpoints/230822 \353\215\260\354\235\264\355\204\260(6, df02)-checkpoint.ipynb"
 create mode 100644 "posts/GNN/FRAUD/.ipynb_checkpoints/230823 \353\215\260\354\235\264\355\204\260(7, df50_com\354\234\274\353\241\234 93\355\215\274 accuracy)_guebin-checkpoint.ipynb"
 create mode 100644 "posts/GNN/FRAUD/.ipynb_checkpoints/230825 \353\215\260\354\235\264\355\204\260(8, df02)\354\273\244\353\204\220\354\243\275\354\235\214.out-checkpoint.ipynb"
 create mode 100644 "posts/GNN/FRAUD/.ipynb_checkpoints/230827 \353\215\260\354\235\264\355\204\260(9, df50 mask\353\247\214\353\223\244\354\227\210\353\212\224\353\215\260 \352\262\260\352\263\274\352\260\222\354\235\264 \353\213\254\353\235\274).out-checkpoint.ipynb"
 delete mode 100644 "posts/GNN/

In [3]:
!git push

Enumerating objects: 14, done.
Counting objects: 100% (14/14), done.
Delta compression using up to 16 threads
Compressing objects: 100% (9/9), done.
Writing objects: 100% (9/9), 5.06 KiB | 5.06 MiB/s, done.
Total 9 (delta 7), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (7/7), completed with 5 local objects.
To https://github.com/boram-coco/coco.git
   1bbb61bf..f9769619  main -> main

In [4]:
!quarto publish gh-pages --no-prompt --no-browser

From https://github.com/boram-coco/coco
 * branch              gh-pages   -> FETCH_HEAD
Rendering for publish:

[  1/182] posts/Python/Summer Program-Python Programming Day1 Quiz.ipynb
[  2/182] posts/Python/4. Class/python 11_0511.ipynb
[  3/182] posts/Python/4. Class/python 13_0530.ipynb
[  4/182] posts/Python/4. Class/python 10_0509.ipynb
[  5/182] posts/Python/4. Class/python 12_0523.ipynb
[  6/182] posts/Python/4. Class/python 13_0525.ipynb
[  7/182] posts/Python/4. Class/python 12_0518.ipynb
[  8/182] posts/Python/4. Class/python 11_0516.ipynb
[  9/182] posts/Python/4. Class/python 14_0606.ipynb
[ 10/182] posts/Python/1. Basic/python 3_0321.ipynb
[ 11/182] posts/Python/1. Basic/python 1_0307.ipynb
[ 12/182] posts/Python/1. Basic/python 4_0323.ipynb
[ 13/182] posts/Python/1. Basic/python 3_0316.ipynb
[ 14/182] posts/Python/1. Basic/python 4_0328.ipynb
[ 15/182] posts/Python/1. Basic/python 2_0314.ipynb
[ 16/182] posts/Python/3. Pandas/python 10_0506 .ipynb
[ 17/182] posts/Python/2